# Tracing Your LLM Application

In this notebook, we are building and instrumenting a simple LLM application that takes a query about art and feeds it into the Metropolitan Museum of Art API to get a list of relevant artwork.

#### Learning goals:
- How to create a span for LLM Observability
- How to annotate a span with inputs and outputs
- Different types of spans, including `tool`, `llm`, and `workflow`.


## Initial Setup


In [12]:
from dotenv import load_dotenv
import os
import requests

from ddtrace.llmobs import LLMObs

load_dotenv()

LLMObs.enable(
    api_key=os.environ.get("DD_API_KEY"),
    site=os.environ.get("DD_SITE", "datadoghq.com"),
    ml_app="test-onboarding-app",
    agentless_enabled=True,
)


The steps to this simple workflow are:

1. Take a question from a user, and parse it into an artwork query using OpenAI.
2. Send the parsed query to the [Metropolitan Museum of Art API](https://metmuseum.github.io/#search).
3. Return a list of URLs to the user.

In [10]:
# Useful constants

SEARCH_ENDPOINT = "https://collectionapi.metmuseum.org/public/collection/v1/search"
MAX_RESULTS = 5

# https://metmuseum.github.io/#search
fetch_met_urls_schema = {
    "type": "function",
    "function": {
        "name": "fetch_met_urls",
        "description": "Submits a query to the MET API and returns urls of relevant artworks",
        "parameters": {
            "type": "object",
            "properties": {
                "query_parameters": {
                    "type": "object",
                    "properties": {
                        "q": {
                            "type": "string",
                            "description": "Represents the users query. Required. Add as many search terms from the query as you can. 'medieval portraits', 'french impressionist paintings', etc.",
                        },
                        "title": {
                            "type": "boolean",
                            "description": "Limits the query to only apply to the title field.",
                        },
                        "tags": {
                            "type": "boolean",
                            "description": "Limits the query to only apply to the tags field.",
                        },
                        "isOnView": {
                            "type": "boolean",
                            "description": "Returns objects that match the query and are on view in the museum.",
                        },
                        "artistOrCulture": {
                            "type": "boolean",
                            "description": "Returns objects that match the query, specifically searching against the artist name or culture field for objects.",
                        },
                        "medium": {
                            "type": "string",
                            "description": 'Returns objects that match the query and are of the specified medium or object type. Examples include: "Ceramics", "Furniture", "Paintings", "Sculpture", "Textiles", etc.',
                        },
                        "geoLocation": {
                            "type": "string",
                            "description": 'Returns objects that match the query and the specified geographic location. Examples include: "Europe", "France", "Paris", "China", "New York", etc.',
                        },
                        "dateBegin": {
                            "type": "number",
                            "description": "You must use both dateBegin and dateEnd, or neither. Returns objects that match the query and fall between the dateBegin and dateEnd parameters. Examples include: dateBegin=1700&dateEnd=1800 for objects from 1700 A.D. to 1800 A.D., dateBegin=-100&dateEnd=100 for objects between 100 B.C. to 100 A.D.",
                        },
                        "dateEnd": {
                            "type": "number",
                            "description": "You must use both dateBegin and dateEnd, or neither. Returns objects that match the query and fall between the dateBegin and dateEnd parameters. Examples include: dateBegin=1700&dateEnd=1800 for objects from 1700 A.D. to 1800 A.D., dateBegin=-100&dateEnd=100 for objects between 100 B.C. to 100 A.D.",
                        },
                    },
                    "required": ["q"],
                },
            },
        },
    },
}

## 1. Tracing tool spans

Below is a simple tool `fetch_met_urls()` to query the Met API's `/search` endpoint. We'll be instrumenting this function, as we can't rely on auto-instrumentation to capture this tool.

In [6]:
def fetch_met_urls(query_parameters):
    response = requests.get(SEARCH_ENDPOINT, params=query_parameters)
    response.raise_for_status()
    object_ids = response.json().get("objectIDs")
    objects_to_return = object_ids[:MAX_RESULTS] if object_ids else []
    urls = [f"https://www.metmuseum.org/art/collection/search/{objectId}" for objectId in objects_to_return]
    return urls

1. **Trace the function**: we can instrument this `fetch_met_urls()` function as a tool span, which represents a call to an external web API (The Met API). We do this by importing from `ddtrace.llmobs.decorators` with our desired span decorator, which is `@tool()`, and applying that decorator over our `fetch_met_urls()` function.

Learn more about tool spans and span kinds in our [docs](https://docs.datadoghq.com/tracing/llm_observability/span_kinds/).

In [24]:
# Import the tool decorator and decorate your tool function.

def fetch_met_urls(query_parameters):
    response = requests.get(SEARCH_ENDPOINT, params=query_parameters)
    response.raise_for_status()
    object_ids = response.json().get("objectIDs")
    objects_to_return = object_ids[:MAX_RESULTS] if object_ids else []
    urls = [f"https://www.metmuseum.org/art/collection/search/{objectId}" for objectId in objects_to_return]
    return urls

2. **Annotate the span**: we now have a span that covers that function's execution, but we can add some extra information to the span to be more useful to us. Let's use `LLMObs.annotate()` to capture the inputs and outputs to this query operation.

*Hint*: The `LLMObs.annotate()` method takes in the following arguments: `input_data`, `output_data`, `metrics`, `metadata`, and `tags`.

Learn more about annotating spans in our [docs](https://docs.datadoghq.com/tracing/llm_observability/sdk/#annotating-a-span).

In [27]:
def fetch_met_urls(query_parameters):
    # We annotate the tool call with input_data here

    response = requests.get(SEARCH_ENDPOINT, params=query_parameters)
    response.raise_for_status()
    object_ids = response.json().get("objectIDs")
    objects_to_return = object_ids[:MAX_RESULTS] if object_ids else []
    urls = [f"https://www.metmuseum.org/art/collection/search/{objectId}" for objectId in objects_to_return]
    # We annotate the tool call with output_data here

    return urls

## 2. Auto-instrumented LLM call

Once again, we are using OpenAI, which is automatically instrumented, so no further annotation is required for `parse_query()`:


In [13]:
from openai import OpenAI
import json

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

system_prompt = """
Example query inputs and outputs for the fetch_met_urls function:

query: medieval french tapestry painting
output: {'q': 'medieval french tapestry painting', geoLocation: 'France', medium: 'Textiles', dateBegin: 1000, dateEnd: 1500}

query: etruscan urns
output: {'q': 'etruscan urn', geoLocation: 'Italy', medium: 'Travertine'}

query: Cambodian hats from the 18th and 19th centuries
output: {'q': 'Cambodian hats', geolocation: 'Cambodia', 'dateBegin': 1700, 'dateEnd': 1900}

"""


def parse_query(message):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message},
    ]
    response_message = (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            tools=[fetch_met_urls_schema],
            # https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice
            tool_choice={"type": "function", "function": {"name": "fetch_met_urls"}},
        )
        .choices[0]
        .message
    )
    if response_message.tool_calls:
        arguments = json.loads(response_message.tool_calls[0].function.arguments)
    return arguments["query_parameters"]

### 3. Tracing workflow spans

Finally, we create a `find_artworks` function here ties the `fetch_met_urls()` tool and `parse_query()` LLM call together. Since this operation involves a sequence of operations which include LLM calls and any supporting operations, we can call this a **workflow**.


In [14]:
def find_artworks(question):
    query = parse_query(question)
    print("Parsed query parameters", query)
    urls = fetch_met_urls(query)
    return urls

1. **Trace the function**: we can instrument this function as a workflow span, similar to how we traced `fetch_met_urls()`.

Learn more about workflow spans in our [docs](https://docs.datadoghq.com/tracing/llm_observability/sdk/#workflow-span).

In [26]:
# Import the workflow decorator and decorate your workflow function.


def find_artworks(question):
    query = parse_query(question)
    print("Parsed query parameters", query)
    urls = fetch_met_urls(query)
    return urls

2. **Annotate your span**: like before, let's annotate the workflow span so that we can see the inputs and outputs of your LLM application on a more granular level.

In [28]:
def find_artworks(question):
    # We annotate the workflow span with input_data here
    query = parse_query(question)
    print("Parsed query parameters", query)
    urls = fetch_met_urls(query)
    # We annotate the workflow span with output_data here
    return urls

## Run your LLM application

Now that your application is instrumented, let's try running it.

In [11]:
urls = find_artworks("paintings of the french revolution")

Parsed query parameters {'q': 'french revolution', 'medium': 'Paintings', 'dateBegin': 1789, 'dateEnd': 1799}


In [12]:
import pprint

pprint.pp(urls)

['https://www.metmuseum.org/art/collection/search/437885',
 'https://www.metmuseum.org/art/collection/search/436875',
 'https://www.metmuseum.org/art/collection/search/436222',
 'https://www.metmuseum.org/art/collection/search/726543']


## Viewing the trace in Datadog

Now, try checking out the [LLM Observability interface](https://app.datadoghq.com/llm) in Datadog. You should see a trace that describes the workflow we just ran.
